In [ ]:
!pip3 -qq install rusenttokenize
!pip3 -qq install pymorphy2
!pip3 -qq install nltk
!pip3 -qq install natasha

In [ ]:
import pandas as pd
import pymorphy2
import string
import nltk
import re
from rusenttokenize import ru_sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk import word_tokenize
nltk.download('punkt')
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

greeting, goodbye with cosine similarity

In [5]:
test_data = pd.read_csv("test_data.csv", encoding="utf-8")

In [6]:
all_txt_list = test_data['text'].tolist()

In [7]:
dlg_dict_mg = {}
for i in range(0,6):
  dlg_dict_mg['dlg'+str(i)+'_mg'] = (test_data.loc[(test_data["dlg_id"] == i)&(test_data["role"] == 'manager')])['text'].tolist()

dlg_dict_all = {}
for i in range(0,6):
  dlg_dict_all['dlg'+str(i)+'_all'] = (test_data.loc[test_data["dlg_id"] == i])['text'].tolist()  

In [8]:
morph_analyzer = pymorphy2.MorphAnalyzer()

In [9]:
def preprocess_tokenize(text):
    text_preprocessed_tokenized = []      
    for sentence in ru_sent_tokenize(text):    
        clean_words = [word.strip(string.punctuation) for word in word_tokenize(text)] 
        clean_words = [word.lower() for word in clean_words]
        clean_lemmas = [morph_analyzer.parse(word)[0].normal_form for word in clean_words]
        text_preprocessed_tokenized.extend(clean_lemmas)
    return text_preprocessed_tokenized

In [10]:
intents = {"greeting": ["Привет", "Здравствуйте", "Добрый день", "Добрый вечер", "Здрасьте", "Здорова", "Добрый денёк", "День добрый", "Рад видеть", "Доброе утро", "Здравствуй", "Доброго времени", "Здравия желаю", "Салам", "Салют", "Бонжур", "Хелло", "Приветствую", "Добро пожаловать", "Доброго здоровья"], "goodbye": ["Пока", "До свидания", "Счастливо", "Всего доброго", "Всего хорошего", "До скорого", "Увидимся", "Спокойной ночи", "До встречи", "Свяжемся позже", "Прощай", "Встретимся", "До свиданья", "Удачи", "Всего наилучшего", "До связи", "Честь имею", "Разрешите откланяться", "Береги себя", "Хорошего дня", "Хорошего вечера", "Поговорим завтра"]}

In [11]:
train_phrases = []
for dialogue in dlg_dict_mg:
  for phrase in dlg_dict_mg[dialogue]:
      train_phrases.append(phrase)

for intent in intents:
  for phrase in intents[intent]:
      train_phrases.append(phrase)

In [ ]:
vectorizer = CountVectorizer(tokenizer=preprocess_tokenize, ngram_range=(1,2))
vectorizer.fit_transform(train_phrases).toarray()

In [13]:
greetings, goodbyes = [],[]
for dialogue in dlg_dict_mg:
  for phrase in dlg_dict_mg[dialogue]:
      vec1 = vectorizer.transform([phrase])
      for intent in intents:
        for sentence in intents[intent]:
          vec2 = vectorizer.transform([sentence])
          similarity = cosine_similarity(vec1, vec2)
          if similarity > 0.3 and intent == 'greeting':
            if phrase not in greetings: 
              greetings.append(phrase)
          if similarity > 0.3 and intent == 'goodbye':
            if phrase not in goodbyes:
              goodbyes.append(phrase)

In [14]:
ethics_check = []
for text in all_txt_list:
  ethics = ''
  for sample in greetings:
    if sample == text:
      ethics = ethics+('greeting=True')
  for sample in goodbyes:
    if sample == text:
      ethics = ethics+('goodbye=True')
  ethics_check.append(ethics)    

In [53]:
#unique values
greetings, goodbyes

(['Алло здравствуйте', 'Алло дмитрий добрый день'],
 ['Всего хорошего до свидания',
  'Угу да вижу я эту почту хорошо тогда исправлю на эту будем ждать ответа всего хорошего',
  'До свидания',
  'Угу все хорошо да понедельника тогда всего доброго',
  'Во вторник все ну с вами да тогда до вторника до свидания',
  'Ну до свидания хорошего вечера'])

In [15]:
test_data['ethics'] = ethics_check

greeting, goodbye with re

In [49]:
re_greetings, re_goodbyes = [],[]
for dialogue in dlg_dict_mg:
  for phrase in dlg_dict_mg[dialogue]:
    if re.findall(r'(.*дравств.*|.*ривет.*|.*обр.*день.*|.*обр.*вечер.*|.*обр.*вечер.*|.*обро.*пожалов.*)', phrase) != []:
      re_greetings.append(phrase)

for dialogue in dlg_dict_mg:
  for phrase in dlg_dict_mg[dialogue]:
    if re.findall(r'(.*свидания.*|.*пока.*|.*Пока*|.*сего.*доброго.*|.*сего.*хорошего.*|.*орошег.*дня.*|.*орошег.*вечер.*)', phrase) != []:
      re_goodbyes.append(phrase)       

In [43]:
re_greetings

['Алло здравствуйте',
 'Алло здравствуйте',
 'Алло здравствуйте',
 'Алло дмитрий добрый день']

In [50]:
re_goodbyes

['Всего хорошего до свидания',
 'Угу да вижу я эту почту хорошо тогда исправлю на эту будем ждать ответа всего хорошего',
 'До свидания',
 'Угу все хорошо да понедельника тогда всего доброго',
 'Во вторник все ну с вами да тогда до вторника до свидания',
 'Ну до свидания хорошего вечера']

NER

In [16]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)

In [17]:
def proper_names(text):
    propered_names = []      
    for sentence in text:    
      sentence_tokenized = word_tokenize(sentence)
      for word in sentence_tokenized:
        if morph_analyzer.parse(word)[0].score < 0.1 and 'INTJ' not in morph_analyzer.parse(word)[0].tag and len(word) > 3:
          propered_names.append(word)
        elif 'Name' in morph_analyzer.parse(word)[0].tag and len(word) > 3:
          propered_names.append(word)
    return propered_names      

In [18]:
pr_names = proper_names(all_txt_list)
all_txt_propered = []
for sentence in all_txt_list:
  sentence = sentence.lower()
  for word in pr_names:
    if word in sentence:
      sentence = sentence.replace(word, word.capitalize())
  all_txt_propered.append(sentence)

In [19]:
ner_name = []
for text in all_txt_propered:
  text_span = ''
  doc = Doc(text)
  doc.segment(segmenter)
  doc.tag_morph(morph_tagger)
  doc.parse_syntax(syntax_parser)
  doc.tag_ner(ner_tagger)
  if doc.spans:
    for span in doc.spans:
      if span.type == 'PER' and ('компания '+span.text) not in text and ('зовут '+span.text) not in text and (span.text+'зовут ') not in text and ('это '+span.text) not in text:
        text_span = text_span+('person='+span.text+' ')
      if span.type == 'PER' and ('компания '+span.text) not in text and (('зовут '+span.text) in text or (span.text+'зовут ') in text or ('это '+span.text) in text):
        text_span = text_span+('self_name='+span.text+' ')
    ner_name.append(text_span)
  else:
    ner_name.append('')       

In [20]:
test_data['ner_name'] = ner_name

In [21]:
ner_org = []
for text in all_txt_propered:
  text_span = ''
  doc = Doc(text)
  doc.segment(segmenter)
  doc.tag_morph(morph_tagger)
  doc.parse_syntax(syntax_parser)
  doc.tag_ner(ner_tagger)
  if doc.spans:
    for span in doc.spans:
      if span.type == 'PER' and ('компания '+span.text) in text:
        text_span = text_span+('organization='+span.text+' ')
      if span.type == 'ORG':
        text_span = text_span+('organization='+span.text+' ')
    ner_org.append(text_span)
  else:
    ner_org.append('')

In [22]:
test_data['ner_org'] = ner_org

In [23]:
test_data

,dlg_id,line_n,role,text,ethics,ner_name,ner_org
0,0,0,client,Алло,,,
1,0,1,manager,Алло здравствуйте,greeting=True,,
2,0,2,client,Добрый день,,,
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,,self_name=Ангелина,organization=Диджитал
4,0,4,client,Ага,,,
...,...,...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,,,
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,,,
477,5,140,client,Спасибо спасибо,,,
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,,,


In [27]:
annotated_data = test_data.to_csv('annotated_data.csv', index=False) 

greeting

In [24]:
test_data.loc[(test_data['role'] == 'manager')&(test_data['ethics'] == 'greeting=True')]['text'].tolist()

['Алло здравствуйте',
 'Алло здравствуйте',
 'Алло здравствуйте',
 'Алло дмитрий добрый день']

goodbye

In [25]:
test_data.loc[(test_data['role'] == 'manager')&(test_data['ethics'] == 'goodbye=True')]['text'].tolist()

['Всего хорошего до свидания',
 'Угу да вижу я эту почту хорошо тогда исправлю на эту будем ждать ответа всего хорошего',
 'До свидания',
 'Угу все хорошо да понедельника тогда всего доброго',
 'Во вторник все ну с вами да тогда до вторника до свидания',
 'Ну до свидания хорошего вечера']

names

In [26]:
test_data.loc[(test_data['role'] == 'manager')&(test_data['ner_name'] != '')]['ner_name'].tolist()

['self_name=Ангелина ',
 'self_name=Ангелина ',
 'self_name=Ангелина ',
 'person=Дмитрий ',
 'person=Максим ',
 'person=Дмитрий person=Анастасия ',
 'person=Айдар ',
 'self_name=Анастасия ',
 'person=Вячеслав ',
 'person=Максим ',
 'person=Дмитрий ']

organizations

In [198]:
test_data.loc[test_data['ner_org'] != '']['ner_org'].tolist()

['organization=Диджитал ',
 'organization=Диджитал ',
 'organization=Диджитал ',
 'organization=Диджитал бизнес ']

checking the condition

In [221]:
for dialogue in range(0,6):
  if test_data.loc[(test_data["dlg_id"] == dialogue)&(test_data['ethics'] == 'greeting=True')]['text'].tolist() != []:
    if test_data.loc[(test_data["dlg_id"] == dialogue)&(test_data['ethics'] == 'goodbye=True')]['text'].tolist() != []:
      print('Требование о приветствии и прощании в диалоге '+ str(dialogue) + ' соблюдено')
    else:
      print('Требование о приветствии и прощании в диалоге '+ str(dialogue) + ' не соблюдено')  
  else:
    print('Требование о приветствии и прощании в диалоге '+ str(dialogue) + ' не соблюдено')     

Требование о приветствии и прощании в диалоге 0 соблюдено
Требование о приветствии и прощании в диалоге 1 соблюдено
Требование о приветствии и прощании в диалоге 2 не соблюдено
Требование о приветствии и прощании в диалоге 3 соблюдено
Требование о приветствии и прощании в диалоге 4 не соблюдено
Требование о приветствии и прощании в диалоге 5 не соблюдено
